In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import sys
sys.path.append('../')
import datetime


In [2]:
class Linear_interpol(object):
    def __init__(self, data):
        self.x, self.y = data
        
    
    def predict(self, z):
        x = np.array(self.x)
        y = np.array(self.y)
        #Проверяем, не подали ли нам уже известную точку
        if (z==x).any():
            return y[np.where(x==z)][0]
        #Находим ближайшую слева и ближайшую справа от искомой точки и их значения
        try:
            min_idx = np.where(x<z)[0][-1]
            max_idx = np.where(x>z)[0][0]
        except IndexError:
            print("Ошибка, точка {} лежит вне интервала [{},{}]".format(z, x.min(), x.max()))
            return None
        x_min = x[min_idx]
        y_min = y[min_idx]
        x_max = x[max_idx]
        y_max = y[max_idx]
        return y_max + (y_min-y_max) / (x_min-x_max) * (z - x_max)
    
    def plot_predict(self, z):
        k = self.predict(z)
        if k != None:
            plt.figure(figsize=(10, 10))
            plt.plot(self.x, self.y, label = 'data')
            plt.plot(z, k,'*', label = 'predicted')
            plt.legend()
            plt.grid()
            plt.show()

def discount_factors_array(dates):
    data = pd.DataFrame(index = dates)
    data['dates'] = np.nan
    last_erodollar = 100 #Костылецкий
    USD_rates = pd.read_csv('../data/USD rates.csv')
    now = datetime.date.today()
    USD_rates.index = USD_rates.EndDate
    USD_rates = USD_rates['Unnamed: 1']
    USD_rates[USD_rates.index[0]] = last_erodollar
    tmp = np.exp(np.log(USD_rates).diff())
    rates = tmp[tmp.index[2]:tmp.index[-1]]
    rates = 1 / rates
    rates = np.exp(np.log(rates).expanding().sum())
    rates.index = pd.to_datetime(rates.index)
    data.index = pd.to_datetime(data.index)
    for_plotting = pd.concat([rates, data],axis=0).drop(['dates'],axis=1).sort_index()
    for_plotting = pd.DataFrame(for_plotting)

    # for_plotting['idx'] = for_plotting.index
    for_plotting.index = pd.to_datetime(for_plotting.index)
    
    for_plotting['interpolation'] = for_plotting.interpolate(method='time')
    
#     plt.plot(1 / for_plotting['interpolation'][dates]) # Если хотите графичек дисконт значений
    return 1 / for_plotting['interpolation'][dates]
#     print(for_plotting)
    # for_plotting.index = for_plotting['idx'].apply(lambda x: datetime.datetime.timestamp(x)) #// 10 ** 9
    # for_plotting.index = for_plotting['idx']

#     plt.plot(for_plotting['interpolation'])
#     plt.plot(rates)
#     plt.legend(['интерполированные', 'настоящие'])
#     plt.show()
#     plt.show()

In [3]:
# dates_to_calc = pd.DataFrame(

#     {
#         "dates": [np.nan, np.nan, np.nan, np.nan]
#     },
#     index=[datetime.date(2024,6,1), datetime.date(2024,7,1), datetime.date(2024,8,1), datetime.date(2024,10,1)],
# )

dates_to_calc = [datetime.date(2024,6,1), datetime.date(2024,7,1), datetime.date(2024,8,1), datetime.date(2024,10,1), datetime.date(2025,10,1)]
# a = pd.DataFrame(index = data)
# a['dates'] = np.nan
# a

In [4]:
discount_factors_array(dates_to_calc)

2024-06-01    0.986971
2024-07-01    0.986678
2024-08-01    0.986405
2024-10-01    0.985873
2025-10-01    0.985222
Name: interpolation, dtype: float64

In [5]:
discount_factors_array(dates_to_calc)['2024-06-01'] == 0.986971

False

In [6]:
np.array(discount_factors_array(dates_to_calc))

array([0.98697129, 0.98667792, 0.98640478, 0.98587309, 0.98522192])